# Timesheet Overlap Calculation

**Use Case:** A user wants to create a **new timesheet entry** and needs to check it won't overlap with any existing timesheets.

**Fields:** `name`, `date_time` (start), `date_time_end` (end), `user_id`

**Algorithm:** Vectorized interval comparison — two intervals overlap iff `A_start < B_end AND B_start < A_end`. Only checks against the same `user_id`.

In [29]:
import pandas as pd
from datetime import datetime, timedelta
from typing import Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

## 1. Overlap Check Function

Validates a single new timesheet against all existing entries for the same user.

In [30]:
def check_overlap(
    existing_df: pd.DataFrame,
    name: str,
    date_time: str,
    date_time_end: str,
    user_id: str
) -> Tuple[bool, Optional[pd.DataFrame]]:
    """
    Check if a new timesheet entry overlaps with any existing entries for the same user.
    
    Efficient: filters by user_id first, then uses vectorized pandas comparison.
    Only scans entries belonging to the same user — O(m) where m = user's entries.
    
    Parameters:
        existing_df: DataFrame of all existing timesheets [name, date_time, date_time_end, user_id]
        name:           Name/description of the new entry
        date_time:      Start datetime of the new entry
        date_time_end:  End datetime of the new entry
        user_id:        User creating the new entry
    
    Returns:
        (is_valid, conflicts)
        - is_valid: True if no overlap, False if overlap found
        - conflicts: DataFrame of conflicting entries (None if valid)
    """
    new_start = pd.to_datetime(date_time)
    new_end = pd.to_datetime(date_time_end)
    
    if new_start >= new_end:
        raise ValueError(f"Start time ({date_time}) must be before end time ({date_time_end})")
    
    if existing_df.empty:
        return True, None
    
    df = existing_df.copy()
    df['date_time'] = pd.to_datetime(df['date_time'])
    df['date_time_end'] = pd.to_datetime(df['date_time_end'])
    
    # Step 1: Filter to same user only (other users can't conflict)
    user_entries = df[df['user_id'] == user_id]
    
    if user_entries.empty:
        return True, None
    
    # Step 2: Vectorized overlap check
    # Two intervals [A_start, A_end) and [B_start, B_end) overlap iff:
    #   A_start < B_end AND B_start < A_end
    conflicts = user_entries[
        (user_entries['date_time'] < new_end) & 
        (new_start < user_entries['date_time_end'])
    ].copy()
    
    if conflicts.empty:
        return True, None
    
    # Add overlap details
    conflicts['overlap_start'] = conflicts['date_time'].clip(lower=new_start)
    conflicts['overlap_end'] = conflicts['date_time_end'].clip(upper=new_end)
    conflicts['overlap_minutes'] = (
        (conflicts['overlap_end'] - conflicts['overlap_start']).dt.total_seconds() / 60
    ).astype(int)
    
    return False, conflicts[['name', 'date_time', 'date_time_end', 'user_id', 'overlap_start', 'overlap_end', 'overlap_minutes']]

In [31]:
def try_create_timesheet(
    existing_df: pd.DataFrame,
    name: str,
    date_time: str,
    date_time_end: str,
    user_id: str
) -> pd.DataFrame:
    """
    Try to create a new timesheet entry. If no overlap, adds it to the dataframe.
    If overlap found, prints conflicts and returns the original dataframe unchanged.
    
    Returns:
        Updated DataFrame (with new entry if valid, unchanged if conflict)
    """
    print(f"📝 Creating: '{name}' for {user_id}")
    print(f"   Time: {date_time} → {date_time_end}")
    print()
    
    is_valid, conflicts = check_overlap(existing_df, name, date_time, date_time_end, user_id)
    
    if is_valid:
        new_row = pd.DataFrame([{
            'name': name,
            'date_time': date_time,
            'date_time_end': date_time_end,
            'user_id': user_id
        }])
        result = pd.concat([existing_df, new_row], ignore_index=True)
        print(f"   ✅ No overlap — entry created successfully!")
        print(f"   Total entries now: {len(result)}")
        return result
    else:
        print(f"   ❌ OVERLAP DETECTED — cannot create entry!")
        print(f"   Conflicts with {len(conflicts)} existing timesheet(s):")
        print()
        for _, c in conflicts.iterrows():
            print(f"   ⚠️  '{c['name']}' ({c['date_time'].strftime('%Y-%m-%d %H:%M')} → {c['date_time_end'].strftime('%Y-%m-%d %H:%M')})")
            print(f"       Overlap: {c['overlap_minutes']} minutes ({c['overlap_start'].strftime('%H:%M')} - {c['overlap_end'].strftime('%H:%M')})")
        return existing_df

## 2. Existing Timesheets (Simulating a Large Dataset)

A packed schedule across multiple users and days — representing a realistic pool of existing entries.

In [32]:
existing_timesheets = [
    # ═══════════════════════════════════════════════════
    # USER_A — Feb 26 (full workday, tightly packed)
    # ═══════════════════════════════════════════════════
    {'name': 'Morning standup',     'date_time': '2026-02-26 08:00', 'date_time_end': '2026-02-26 08:30', 'user_id': 'user_a'},
    {'name': 'Email & planning',    'date_time': '2026-02-26 08:30', 'date_time_end': '2026-02-26 09:00', 'user_id': 'user_a'},
    {'name': 'Feature dev: Auth',   'date_time': '2026-02-26 09:00', 'date_time_end': '2026-02-26 11:00', 'user_id': 'user_a'},
    {'name': 'Code review: PR #42', 'date_time': '2026-02-26 11:00', 'date_time_end': '2026-02-26 12:00', 'user_id': 'user_a'},
    {'name': 'Lunch break',         'date_time': '2026-02-26 12:00', 'date_time_end': '2026-02-26 13:00', 'user_id': 'user_a'},
    {'name': 'Client meeting',      'date_time': '2026-02-26 13:00', 'date_time_end': '2026-02-26 14:00', 'user_id': 'user_a'},
    {'name': 'Bug fix: JIRA-123',   'date_time': '2026-02-26 14:00', 'date_time_end': '2026-02-26 15:30', 'user_id': 'user_a'},
    {'name': 'Testing & QA',        'date_time': '2026-02-26 15:30', 'date_time_end': '2026-02-26 17:00', 'user_id': 'user_a'},
    
    # USER_A — Feb 27 (partial day)
    {'name': 'Morning standup',     'date_time': '2026-02-27 08:00', 'date_time_end': '2026-02-27 08:30', 'user_id': 'user_a'},
    {'name': 'Sprint planning',     'date_time': '2026-02-27 08:30', 'date_time_end': '2026-02-27 10:00', 'user_id': 'user_a'},
    {'name': 'Feature dev: API',    'date_time': '2026-02-27 10:00', 'date_time_end': '2026-02-27 12:00', 'user_id': 'user_a'},
    {'name': 'Lunch break',         'date_time': '2026-02-27 12:00', 'date_time_end': '2026-02-27 13:00', 'user_id': 'user_a'},
    {'name': 'Documentation',       'date_time': '2026-02-27 13:00', 'date_time_end': '2026-02-27 15:00', 'user_id': 'user_a'},
    {'name': 'Retrospective',       'date_time': '2026-02-27 15:00', 'date_time_end': '2026-02-27 16:00', 'user_id': 'user_a'},
    
    # ═══════════════════════════════════════════════════
    # USER_B — Feb 26 (different schedule, some gaps)
    # ═══════════════════════════════════════════════════
    {'name': 'Design review',       'date_time': '2026-02-26 09:00', 'date_time_end': '2026-02-26 10:30', 'user_id': 'user_b'},
    {'name': 'UI implementation',   'date_time': '2026-02-26 10:30', 'date_time_end': '2026-02-26 12:30', 'user_id': 'user_b'},
    {'name': 'Lunch',               'date_time': '2026-02-26 12:30', 'date_time_end': '2026-02-26 13:30', 'user_id': 'user_b'},
    {'name': 'Pair programming',    'date_time': '2026-02-26 13:30', 'date_time_end': '2026-02-26 15:00', 'user_id': 'user_b'},
    # gap: 15:00 - 16:00 (free for user_b)
    {'name': 'Team sync',           'date_time': '2026-02-26 16:00', 'date_time_end': '2026-02-26 17:00', 'user_id': 'user_b'},
    
    # USER_B — Feb 27
    {'name': 'Standup',             'date_time': '2026-02-27 09:00', 'date_time_end': '2026-02-27 09:30', 'user_id': 'user_b'},
    {'name': 'Frontend refactor',   'date_time': '2026-02-27 09:30', 'date_time_end': '2026-02-27 12:00', 'user_id': 'user_b'},
    {'name': 'Lunch',               'date_time': '2026-02-27 12:00', 'date_time_end': '2026-02-27 13:00', 'user_id': 'user_b'},
    {'name': 'Deployment prep',     'date_time': '2026-02-27 13:00', 'date_time_end': '2026-02-27 15:00', 'user_id': 'user_b'},
    
    # ═══════════════════════════════════════════════════
    # USER_C — Feb 26 (light schedule with gaps)
    # ═══════════════════════════════════════════════════
    {'name': 'Research',            'date_time': '2026-02-26 10:00', 'date_time_end': '2026-02-26 12:00', 'user_id': 'user_c'},
    # gap: 12:00 - 14:00 (free for user_c)
    {'name': 'Prototype build',     'date_time': '2026-02-26 14:00', 'date_time_end': '2026-02-26 16:00', 'user_id': 'user_c'},
    {'name': 'Wrap-up notes',       'date_time': '2026-02-26 16:00', 'date_time_end': '2026-02-26 16:30', 'user_id': 'user_c'},
]

df = pd.DataFrame(existing_timesheets)
print(f"📊 Existing timesheets: {len(df)} entries across {df['user_id'].nunique()} users")
print()

# Show schedule per user
for uid in sorted(df['user_id'].unique()):
    user_df = df[df['user_id'] == uid]
    print(f"  {uid}: {len(user_df)} entries")

print()
df

📊 Existing timesheets: 26 entries across 3 users

  user_a: 14 entries
  user_b: 9 entries
  user_c: 3 entries



,name,date_time,date_time_end,user_id
0,Morning standup,2026-02-26 08:00,2026-02-26 08:30,user_a
1,Email & planning,2026-02-26 08:30,2026-02-26 09:00,user_a
2,Feature dev: Auth,2026-02-26 09:00,2026-02-26 11:00,user_a
3,Code review: PR #42,2026-02-26 11:00,2026-02-26 12:00,user_a
4,Lunch break,2026-02-26 12:00,2026-02-26 13:00,user_a
5,Client meeting,2026-02-26 13:00,2026-02-26 14:00,user_a
6,Bug fix: JIRA-123,2026-02-26 14:00,2026-02-26 15:30,user_a
7,Testing & QA,2026-02-26 15:30,2026-02-26 17:00,user_a
8,Morning standup,2026-02-27 08:00,2026-02-27 08:30,user_a
9,Sprint planning,2026-02-27 08:30,2026-02-27 10:00,user_a


## 3. Test: Creating a New Timesheet (NO Overlap)

These entries should pass validation — they fit into gaps in each user's schedule.

In [33]:
# ═══ TEST 1: user_a has a gap on Feb 26 after 17:00 ═══
print("━" * 65)
print("TEST 1: user_a — evening slot (after work hours)")
print("━" * 65)
df = try_create_timesheet(
    existing_df=df,
    name='Overtime: deploy fix',
    date_time='2026-02-26 17:30',
    date_time_end='2026-02-26 19:00',
    user_id='user_a'
)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
TEST 1: user_a — evening slot (after work hours)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📝 Creating: 'Overtime: deploy fix' for user_a
   Time: 2026-02-26 17:30 → 2026-02-26 19:00

   ✅ No overlap — entry created successfully!
   Total entries now: 27


In [34]:
# ═══ TEST 2: user_b has a gap on Feb 26 between 15:00 and 16:00 ═══
print("━" * 65)
print("TEST 2: user_b — fitting into the 15:00-16:00 gap")
print("━" * 65)
df = try_create_timesheet(
    existing_df=df,
    name='Quick break & coffee',
    date_time='2026-02-26 15:00',
    date_time_end='2026-02-26 16:00',
    user_id='user_b'
)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
TEST 2: user_b — fitting into the 15:00-16:00 gap
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📝 Creating: 'Quick break & coffee' for user_b
   Time: 2026-02-26 15:00 → 2026-02-26 16:00

   ✅ No overlap — entry created successfully!
   Total entries now: 28


In [35]:
# ═══ TEST 3: user_c has a gap on Feb 26 between 12:00 and 14:00 ═══
print("━" * 65)
print("TEST 3: user_c — fitting into the 12:00-14:00 gap")
print("━" * 65)
df = try_create_timesheet(
    existing_df=df,
    name='Lunch & reading',
    date_time='2026-02-26 12:00',
    date_time_end='2026-02-26 14:00',
    user_id='user_c'
)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
TEST 3: user_c — fitting into the 12:00-14:00 gap
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📝 Creating: 'Lunch & reading' for user_c
   Time: 2026-02-26 12:00 → 2026-02-26 14:00

   ✅ No overlap — entry created successfully!
   Total entries now: 29


In [36]:
# ═══ TEST 4: completely new day for user_a ═══
print("━" * 65)
print("TEST 4: user_a — new day with no entries (Feb 28)")
print("━" * 65)
df = try_create_timesheet(
    existing_df=df,
    name='Workshop training',
    date_time='2026-02-28 09:00',
    date_time_end='2026-02-28 12:00',
    user_id='user_a'
)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
TEST 4: user_a — new day with no entries (Feb 28)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📝 Creating: 'Workshop training' for user_a
   Time: 2026-02-28 09:00 → 2026-02-28 12:00

   ✅ No overlap — entry created successfully!
   Total entries now: 30


## 4. Test: Creating a New Timesheet (OVERLAP — Should Fail)

These entries should be **rejected** because they conflict with existing timesheets.

In [37]:
# ═══ TEST 5: Overlaps with user_a's "Feature dev: Auth" (09:00-11:00) ═══
print("━" * 65)
print("TEST 5: user_a — overlaps with Feature dev (09:00-11:00)")
print("━" * 65)
df = try_create_timesheet(
    existing_df=df,
    name='Urgent call with PM',
    date_time='2026-02-26 10:00',
    date_time_end='2026-02-26 10:30',
    user_id='user_a'
)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
TEST 5: user_a — overlaps with Feature dev (09:00-11:00)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📝 Creating: 'Urgent call with PM' for user_a
   Time: 2026-02-26 10:00 → 2026-02-26 10:30

   ❌ OVERLAP DETECTED — cannot create entry!
   Conflicts with 1 existing timesheet(s):

   ⚠️  'Feature dev: Auth' (2026-02-26 09:00 → 2026-02-26 11:00)
       Overlap: 30 minutes (10:00 - 10:30)


In [38]:
# ═══ TEST 6: Spans across MULTIPLE existing entries for user_a ═══
print("━" * 65)
print("TEST 6: user_a — spans across multiple entries (10:30-13:30)")
print("━" * 65)
df = try_create_timesheet(
    existing_df=df,
    name='Long workshop',
    date_time='2026-02-26 10:30',
    date_time_end='2026-02-26 13:30',
    user_id='user_a'
)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
TEST 6: user_a — spans across multiple entries (10:30-13:30)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📝 Creating: 'Long workshop' for user_a
   Time: 2026-02-26 10:30 → 2026-02-26 13:30

   ❌ OVERLAP DETECTED — cannot create entry!
   Conflicts with 4 existing timesheet(s):

   ⚠️  'Feature dev: Auth' (2026-02-26 09:00 → 2026-02-26 11:00)
       Overlap: 30 minutes (10:30 - 11:00)
   ⚠️  'Code review: PR #42' (2026-02-26 11:00 → 2026-02-26 12:00)
       Overlap: 60 minutes (11:00 - 12:00)
   ⚠️  'Lunch break' (2026-02-26 12:00 → 2026-02-26 13:00)
       Overlap: 60 minutes (12:00 - 13:00)
   ⚠️  'Client meeting' (2026-02-26 13:00 → 2026-02-26 14:00)
       Overlap: 30 minutes (13:00 - 13:30)


In [39]:
# ═══ TEST 7: Exact same time as an existing entry ═══
print("━" * 65)
print("TEST 7: user_b — exact duplicate of 'Design review' (09:00-10:30)")
print("━" * 65)
df = try_create_timesheet(
    existing_df=df,
    name='Another meeting',
    date_time='2026-02-26 09:00',
    date_time_end='2026-02-26 10:30',
    user_id='user_b'
)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
TEST 7: user_b — exact duplicate of 'Design review' (09:00-10:30)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📝 Creating: 'Another meeting' for user_b
   Time: 2026-02-26 09:00 → 2026-02-26 10:30

   ❌ OVERLAP DETECTED — cannot create entry!
   Conflicts with 1 existing timesheet(s):

   ⚠️  'Design review' (2026-02-26 09:00 → 2026-02-26 10:30)
       Overlap: 90 minutes (09:00 - 10:30)


In [40]:
# ═══ TEST 8: End time bleeds into next entry by 1 minute ═══
print("━" * 65)
print("TEST 8: user_c — ends 1 min into 'Prototype build' (14:00-16:00)")
print("━" * 65)
df = try_create_timesheet(
    existing_df=df,
    name='Quick sync call',
    date_time='2026-02-26 13:30',
    date_time_end='2026-02-26 14:01',
    user_id='user_c'
)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
TEST 8: user_c — ends 1 min into 'Prototype build' (14:00-16:00)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📝 Creating: 'Quick sync call' for user_c
   Time: 2026-02-26 13:30 → 2026-02-26 14:01

   ❌ OVERLAP DETECTED — cannot create entry!
   Conflicts with 2 existing timesheet(s):

   ⚠️  'Prototype build' (2026-02-26 14:00 → 2026-02-26 16:00)
       Overlap: 1 minutes (14:00 - 14:01)
   ⚠️  'Lunch & reading' (2026-02-26 12:00 → 2026-02-26 14:00)
       Overlap: 30 minutes (13:30 - 14:00)


In [41]:
# ═══ TEST 9: Entry that wraps around an existing one entirely ═══
print("━" * 65)
print("TEST 9: user_c — wraps around 'Research' (10:00-12:00)")
print("━" * 65)
df = try_create_timesheet(
    existing_df=df,
    name='All-day meeting',
    date_time='2026-02-26 08:00',
    date_time_end='2026-02-26 18:00',
    user_id='user_c'
)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
TEST 9: user_c — wraps around 'Research' (10:00-12:00)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📝 Creating: 'All-day meeting' for user_c
   Time: 2026-02-26 08:00 → 2026-02-26 18:00

   ❌ OVERLAP DETECTED — cannot create entry!
   Conflicts with 4 existing timesheet(s):

   ⚠️  'Research' (2026-02-26 10:00 → 2026-02-26 12:00)
       Overlap: 120 minutes (10:00 - 12:00)
   ⚠️  'Prototype build' (2026-02-26 14:00 → 2026-02-26 16:00)
       Overlap: 120 minutes (14:00 - 16:00)
   ⚠️  'Wrap-up notes' (2026-02-26 16:00 → 2026-02-26 16:30)
       Overlap: 30 minutes (16:00 - 16:30)
   ⚠️  'Lunch & reading' (2026-02-26 12:00 → 2026-02-26 14:00)
       Overlap: 120 minutes (12:00 - 14:00)


## 5. Edge Case: Different Users, Same Time = OK

Overlap checks are **per-user**. Different users can book the same time slot.

In [42]:
# ═══ TEST 10: Different user at the same time as user_a's entry — should PASS ═══
print("━" * 65)
print("TEST 10: user_d — same time as user_a's slot (should pass!)")
print("━" * 65)
df = try_create_timesheet(
    existing_df=df,
    name='Independent work',
    date_time='2026-02-26 09:00',
    date_time_end='2026-02-26 11:00',
    user_id='user_d'
)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
TEST 10: user_d — same time as user_a's slot (should pass!)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📝 Creating: 'Independent work' for user_d
   Time: 2026-02-26 09:00 → 2026-02-26 11:00

   ✅ No overlap — entry created successfully!
   Total entries now: 31


## 6. Final State — All Timesheets

In [43]:
print(f"📊 Final state: {len(df)} total entries")
print()
for uid in sorted(df['user_id'].unique()):
    user_df = df[df['user_id'] == uid].copy()
    user_df['date_time'] = pd.to_datetime(user_df['date_time'])
    user_df = user_df.sort_values('date_time')
    print(f"\n{'═' * 50}")
    print(f"  {uid} ({len(user_df)} entries)")
    print(f"{'═' * 50}")
    for _, row in user_df.iterrows():
        start = pd.to_datetime(row['date_time']).strftime('%Y-%m-%d %H:%M')
        end = pd.to_datetime(row['date_time_end']).strftime('%H:%M')
        print(f"  {start} - {end}  │ {row['name']}")

📊 Final state: 31 total entries


══════════════════════════════════════════════════
  user_a (16 entries)
══════════════════════════════════════════════════
  2026-02-26 08:00 - 08:30  │ Morning standup
  2026-02-26 08:30 - 09:00  │ Email & planning
  2026-02-26 09:00 - 11:00  │ Feature dev: Auth
  2026-02-26 11:00 - 12:00  │ Code review: PR #42
  2026-02-26 12:00 - 13:00  │ Lunch break
  2026-02-26 13:00 - 14:00  │ Client meeting
  2026-02-26 14:00 - 15:30  │ Bug fix: JIRA-123
  2026-02-26 15:30 - 17:00  │ Testing & QA
  2026-02-26 17:30 - 19:00  │ Overtime: deploy fix
  2026-02-27 08:00 - 08:30  │ Morning standup
  2026-02-27 08:30 - 10:00  │ Sprint planning
  2026-02-27 10:00 - 12:00  │ Feature dev: API
  2026-02-27 12:00 - 13:00  │ Lunch break
  2026-02-27 13:00 - 15:00  │ Documentation
  2026-02-27 15:00 - 16:00  │ Retrospective
  2026-02-28 09:00 - 12:00  │ Workshop training

══════════════════════════════════════════════════
  user_b (10 entries)
═══════════════════════════════

## 7. Performance Notes

| Operation | Complexity | Description |
|---|---|---|
| `check_overlap()` | **O(m)** | Filter by user_id + vectorized comparison, m = user's entries |
| `try_create_timesheet()` | **O(m)** | Calls check_overlap + append if valid |
| Naive approach | O(n) | Would scan all entries across all users |

### Key efficiency points:
- **Filter by `user_id` first** — only checks entries belonging to the same user
- **Vectorized comparison** — pandas boolean indexing, no Python loops
- **Single formula** — `A_start < B_end AND B_start < A_end` covers all overlap types (partial, nested, exact)